Importing required libraries

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.preprocessing import StandardScaler

ModuleNotFoundError: No module named 'pandas'

Loading the data

In [ ]:
train_df = pd.read_csv('kdd_train_final.csv')
test_df = pd.read_csv('kdd_test_final.csv')

In [ ]:
train_df.head()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,protocol_type_encoded,service_encoded,flag_encoded,attack_binary
0,0.0,491.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.17,0.00,0.00,0.00,0.05,0.00,1.0,20.0,9.0,0
1,0.0,146.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.88,0.00,0.00,0.00,0.00,0.00,2.0,44.0,9.0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,1.00,1.00,0.00,0.00,1.0,49.0,5.0,1
3,0.0,232.0,8153.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.03,0.04,0.03,0.01,0.00,0.01,1.0,24.0,9.0,0
4,0.0,199.0,420.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,1.0,24.0,9.0,0


In [ ]:
TARGET_COLUMN = 'attack_binary' 
FEATURE_COLUMNS = [col for col in train_df.columns if col != TARGET_COLUMN]

Splitting the data

In [ ]:
X_train = train_df[FEATURE_COLUMNS]
y_train = train_df[TARGET_COLUMN]
X_test = test_df[FEATURE_COLUMNS]
y_test = test_df[TARGET_COLUMN]

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
print(f"Data loaded and prepared for {len(FEATURE_COLUMNS)} features and target '{TARGET_COLUMN}'.\n")

Data loaded and prepared for 41 features and target 'attack_binary'.



Creating a dictionary to save all the models

In [ ]:
models = {
    "Linear Regression (OLS)": LinearRegression(), 
    "Gradient Descent (SGD)": SGDRegressor(max_iter=1000, tol=1e-3, random_state=42), 
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42)
}

In [ ]:
results = {}

In [ ]:
for name, model in models.items():
    print(f"--- Training {name} ---")
    
    if name == "Gradient Descent (SGD)":
        model.fit(X_train_scaled, y_train)
        predictions = model.predict(X_test_scaled)
    else:
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)

--- Training Linear Regression (OLS) ---
--- Training Gradient Descent (SGD) ---
--- Training Decision Tree ---
--- Training Random Forest ---
